In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import sys
import warnings
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

warnings.filterwarnings('ignore')

project_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.insert(0, project_root)

In [2]:
# Constants
REBRANDING_DATE = pd.to_datetime('2025-05-01')
HOT_SEASON_MONTHS = [4, 5, 6, 7, 8, 9, 10]  # April-October (Indonesia)
RAINY_SEASON_MONTHS = [11, 12, 1, 2, 3]  # November-March (Indonesia)
FORECAST_DAYS = 90

In [3]:
class ProductionForecaster:
    """
    Production model focused on performance evaluation (RF & GB only).
    """
    
    def __init__(self, model_type='rf'):
        self.model_type = model_type
        self.model = self._create_model()
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self.category_encoder = LabelEncoder()
        self.feature_columns = []
        self.items = []
        self.item_categories = {}
        
    def _create_model(self):
        """Create the model architecture"""
        if self.model_type == 'rf':
            return RandomForestRegressor(
                n_estimators=200,
                max_depth=15,
                min_samples_split=5,
                min_samples_leaf=2,
                max_features=1.0,
                random_state=42,
                n_jobs=-1
            )
        elif self.model_type == 'gb':
            return GradientBoostingRegressor(
                n_estimators=200,
                learning_rate=0.1,
                max_depth=6,
                subsample=0.8,
                random_state=42
            )
        else:
            return RandomForestRegressor(random_state=42) # Default
    
    def create_advanced_categories(self, df):
        """Create item categories for feature engineering"""
        categories = {
            'Coffee_Hot': ['kopi', 'cappucino', 'tubruk', 'drip', 'espresso'],
            'Coffee_Cold': ['vietnam', 'ice', 'cold', 'frappe'],
            'Tea': ['tea', 'teh'],
            'Lemon': ['lemon'],
            'Milk_Dairy': ['milk', 'susu', 'latte'],
            'Food_Main': ['nasi', 'mie', 'ayam', 'sapi'],
            'Food_Snack': ['kentang', 'cireng', 'basreng', 'tahu', 'tempe'],
            'Food_Fried': ['goreng', 'fried'],
            'Juice': ['juice', 'jus'],
            'Other': []
        }
        
        for item in df['Item'].unique():
            item_lower = item.lower()
            categorized = False
            for category, keywords in categories.items():
                if category == 'Other': continue
                if any(keyword in item_lower for keyword in keywords):
                    self.item_categories[item] = category
                    categorized = True
                    break
            if not categorized:
                self.item_categories[item] = 'Other'
        return self.item_categories

    def _create_date_features(self, date_series):
        """Standardized date features"""
        date_features = {}
        date_features['day_of_week'] = date_series.dt.dayofweek
        date_features['day_of_month'] = date_series.dt.day
        date_features['month'] = date_series.dt.month
        date_features['quarter'] = date_series.dt.quarter
        date_features['year'] = date_series.dt.year
        date_features['week_of_year'] = date_series.dt.isocalendar().week
        date_features['day_of_year'] = date_series.dt.dayofyear
        
        date_features['is_weekend'] = (date_features['day_of_week'] >= 5).astype(int)
        date_features['is_monday'] = (date_features['day_of_week'] == 0).astype(int)
        date_features['is_friday'] = (date_features['day_of_week'] == 4).astype(int)
        
        # Indonesia Seasons
        HOT_SEASON = [4, 5, 6, 7, 8, 9, 10]
        RAINY_SEASON = [11, 12, 1, 2, 3]
        date_features['is_hot_season'] = date_features['month'].isin(HOT_SEASON).astype(int)
        date_features['is_rainy_season'] = date_features['month'].isin(RAINY_SEASON).astype(int)
        
        date_features['is_month_start'] = (date_features['day_of_month'] <= 3).astype(int)
        date_features['is_month_end'] = (date_features['day_of_month'] >= 28).astype(int)
        
        # Structural break
        REBRANDING_DATE = pd.to_datetime('2025-05-01')
        date_features['post_rebranding'] = (date_series >= REBRANDING_DATE).astype(int)
        
        return date_features

    def prepare_production_features(self, df):
        """Feature engineering pipeline"""
        df = df.copy()
        df['Date'] = pd.to_datetime(df['Date'])
        
        # Date Features
        date_features = self._create_date_features(df['Date'])
        for k, v in date_features.items(): df[k] = v
        
        # Categories
        if not self.item_categories: self.create_advanced_categories(df)
        df['category'] = df['Item'].map(self.item_categories)
        
        # Sort for lags
        df = df.sort_values(['Item', 'Date'])
        
        # Lags & Rolling
        grp = df.groupby('Item')['Quantity_Sold']
        df['quantity_lag_1'] = grp.shift(1)
        df['quantity_lag_7'] = grp.shift(7)
        df['quantity_lag_14'] = grp.shift(14)
        df['quantity_lag_30'] = grp.shift(30)
        
        df['rolling_mean_3'] = grp.transform(lambda x: x.rolling(3, min_periods=1).mean())
        df['rolling_std_3'] = grp.transform(lambda x: x.rolling(3, min_periods=1).std())
        df['rolling_mean_7'] = grp.transform(lambda x: x.rolling(7, min_periods=1).mean())
        df['rolling_std_7'] = grp.transform(lambda x: x.rolling(7, min_periods=1).std())
        df['rolling_mean_30'] = grp.transform(lambda x: x.rolling(30, min_periods=1).mean())
        df['ewm_7'] = grp.transform(lambda x: x.ewm(span=7, adjust=False).mean())
        
        # Trend
        df['trend_7'] = df['Quantity_Sold'] - df['rolling_mean_7']
        
        # Item Stats
        item_stats = df.groupby('Item')['Quantity_Sold'].agg(['mean', 'std', 'min', 'max']).reset_index()
        item_stats.columns = ['Item', 'item_mean', 'item_std', 'item_min', 'item_max']
        df = df.merge(item_stats, on='Item', how='left')
        
        # Encodings
        df['item_encoded'] = self.label_encoder.fit_transform(df['Item'])
        df['category_encoded'] = self.category_encoder.fit_transform(df['category'])
        
        # Volatility
        df['quantity_volatility'] = df['item_std'] / (df['item_mean'] + 1e-8)
        
        return df

    def train(self, df, target_col='Quantity_Sold'):
        """Train model"""
        df_features = self.prepare_production_features(df)
        self.items = df['Item'].unique()
        
        feature_cols = [
            'item_encoded', 'category_encoded', 'day_of_week', 'day_of_month', 'month', 
            'is_weekend', 'is_hot_season', 'post_rebranding',
            'quantity_lag_1', 'quantity_lag_7', 'quantity_lag_14', 'quantity_lag_30',
            'rolling_mean_3', 'rolling_std_3', 'rolling_mean_7', 'rolling_std_7', 
            'rolling_mean_30', 'ewm_7', 'trend_7',
            'item_mean', 'item_std', 'item_min', 'item_max', 'quantity_volatility'
        ]
        
        # Filter available columns
        self.feature_columns = [c for c in feature_cols if c in df_features.columns]
        
        train_data = df_features.dropna(subset=self.feature_columns + [target_col])
        X = train_data[self.feature_columns]
        y = train_data[target_col].round().astype(int)
        
        if self.model is not None:
            self.model.fit(X, y)
            
        return X, y

    def predict(self, X):
        """Predict on feature matrix"""
        if self.model is None: return np.zeros(len(X), dtype=int)
        
        X_clean = pd.DataFrame(X).fillna(0)
        y_pred = self.model.predict(X_clean)
        return np.maximum(np.round(y_pred), 0).astype(int)

    def evaluate_on_split(self, train_df, test_df):
        """Train on train_df, evaluate on test_df, return metrics"""
        # Train
        self.train(train_df)
        
        # Prepare Test
        test_feat = self.prepare_production_features(test_df)
        test_clean = test_feat.dropna(subset=self.feature_columns + ['Quantity_Sold'])
        
        if len(test_clean) == 0:
            return None
            
        X_test = test_clean[self.feature_columns]
        y_test = test_clean['Quantity_Sold'].round().astype(int)
        
        # Predict
        y_pred = self.predict(X_test)
        
        # Metrics
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mape = mean_absolute_percentage_error(y_test, y_pred) * 100
        
        return {
            'mae': mae, 
            'rmse': rmse, 
            'mape': mape, 
            'y_pred': y_pred,
            'y_true': y_test.values,
            'test_indices': test_clean.index
        }

    def get_feature_importance(self):
        """Return feature importance DataFrame"""
        if hasattr(self.model, 'feature_importances_'):
            imps = self.model.feature_importances_
            return pd.DataFrame({
                'Feature': self.feature_columns,
                'Importance': imps
            }).sort_values('Importance', ascending=False)
        return pd.DataFrame()

In [4]:
class ProductionEnsembleForecaster:
    """Evaluation-only Ensemble (RF + GB)"""
    
    def __init__(self):
        self.rf = ProductionForecaster('rf')
        self.gb = ProductionForecaster('gb')
        
    def evaluate_ensemble(self, train_df, test_df):
        """Train both, predict both, average results, calculate metrics"""
        print("   > Training RF...")
        rf_res = self.rf.evaluate_on_split(train_df, test_df)
        
        print("   > Training GB...")
        gb_res = self.gb.evaluate_on_split(train_df, test_df)
        
        if rf_res is None or gb_res is None:
            return None
            
        # Ensemble Averaging
        pred_rf = rf_res['y_pred']
        pred_gb = gb_res['y_pred']
        
        # Simple Average Ensemble
        ensemble_pred = np.round((pred_rf + pred_gb) / 2).astype(int)
        y_true = rf_res['y_true'] # Same for both
        
        # Metrics
        mae = mean_absolute_error(y_true, ensemble_pred)
        rmse = np.sqrt(mean_squared_error(y_true, ensemble_pred))
        mape = mean_absolute_percentage_error(y_true, ensemble_pred) * 100
        
        return {
            'rf_metrics': rf_res,
            'gb_metrics': gb_res,
            'ensemble_metrics': {'mae': mae, 'rmse': rmse, 'mape': mape},
            'rf_importance': self.rf.get_feature_importance()
        }

In [5]:
def run_performance_evaluation():
    """Run comprehensive model performance evaluation"""
    print("="*60)
    print("MODEL PERFORMANCE EVALUATION (RF vs GB vs ENSEMBLE)")
    print("="*60)
    
    # 1. Load Data
    base_path = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
    data_path = os.path.join(base_path, 'data', 'processed', 'daily_item_sales.csv')
    
    if not os.path.exists(data_path):
        print("Error: Data file not found.")
        return
        
    df = pd.read_csv(data_path)
    df['Date'] = pd.to_datetime(df['Date'])
    
    # 2. Split Data (Train / Test)
    # Strategy: Use last 60 days as Holdout Test Set
    cutoff_date = df['Date'].max() - timedelta(days=60)
    
    train_df = df[df['Date'] <= cutoff_date].copy()
    test_df = df[df['Date'] > cutoff_date].copy()
    
    print(f"Total Records: {len(df)}")
    print(f"Train Set:     {len(train_df)} records (Up to {cutoff_date.date()})")
    print(f"Test Set:      {len(test_df)} records (Last 60 Days)")
    print("-" * 60)
    
    # 3. Run Evaluation
    ensemble = ProductionEnsembleForecaster()
    results = ensemble.evaluate_ensemble(train_df, test_df)
    
    if results is None:
        print("Evaluation failed (insufficient data for features in test set).")
        return

    # 4. Print Summary Table
    rf = results['rf_metrics']
    gb = results['gb_metrics']
    ens = results['ensemble_metrics']
    
    print("\n" + "="*45)
    print("PERFORMANCE METRICS SUMMARY (Test Set)")
    print("="*45)
    print(f"{'Metric':<10} | {'RF':<10} | {'GB':<10} | {'ENSEMBLE':<10}")
    print("-" * 45)
    print(f"{'MAE':<10} | {rf['mae']:<10.3f} | {gb['mae']:<10.3f} | {ens['mae']:<10.3f}")
    print(f"{'RMSE':<10} | {rf['rmse']:<10.3f} | {gb['rmse']:<10.3f} | {ens['rmse']:<10.3f}")
    print(f"{'MAPE':<10} | {rf['mape']:<10.2f}% | {gb['mape']:<10.2f}% | {ens['mape']:<10.2f}%")
    print("-" * 45)
    
    best_model = "Ensemble"
    if rf['mae'] < ens['mae'] and rf['mae'] < gb['mae']: best_model = "Random Forest"
    if gb['mae'] < ens['mae'] and gb['mae'] < rf['mae']: best_model = "Gradient Boosting"
    
    print(f"\n🏆 Best Performing Model: {best_model}")

    # 5. Feature Importance Analysis (RF)
    print("\n" + "="*45)
    print("TOP 10 DRIVERS OF ACCURACY (RF Importance)")
    print("="*45)
    imp_df = results['rf_importance']
    if not imp_df.empty:
        print(imp_df.head(10).to_string(index=False))
    else:
        print("Feature importance not available.")

# Execute
run_performance_evaluation()

MODEL PERFORMANCE EVALUATION (RF vs GB vs ENSEMBLE)
Total Records: 32447
Train Set:     30408 records (Up to 2025-07-27)
Test Set:      2039 records (Last 60 Days)
------------------------------------------------------------
   > Training RF...
   > Training GB...

PERFORMANCE METRICS SUMMARY (Test Set)
Metric     | RF         | GB         | ENSEMBLE  
---------------------------------------------
MAE        | 0.012      | 0.000      | 0.012     
RMSE       | 0.109      | 0.000      | 0.109     
MAPE       | 0.12      % | 0.00      % | 0.12      %
---------------------------------------------

🏆 Best Performing Model: Gradient Boosting

TOP 10 DRIVERS OF ACCURACY (RF Importance)
        Feature  Importance
        trend_7    0.734002
  rolling_std_3    0.071944
  rolling_std_7    0.068865
 rolling_mean_7    0.065041
          ewm_7    0.050888
 rolling_mean_3    0.007530
       item_std    0.000386
       item_max    0.000385
rolling_mean_30    0.000146
    day_of_week    0.000133
